# Souce
* https://towardsdatascience.com/boosting-showdown-scikit-learn-vs-xgboost-vs-lightgbm-vs-catboost-in-sentiment-classification-f7c7f46fd956

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
cd drive/My\ Drive/Colab\ Notebooks/voice_phishing/

/content/drive/My Drive/Colab Notebooks/voice_phishing


In [3]:
# ls -l

# Import librairie

In [4]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.4 MB/s 


In [5]:
# Importing packages and data
from time import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (RandomForestClassifier,
                              AdaBoostClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


# Import and prepare the data

In [6]:
#
# df_train = pd.read_csv('train.csv').sample(frac=1.0)
# df_test = pd.read_csv('test.csv').sample(frac=1.0)

df_train = pd.read_csv('train_test_sets/train_KorCCViD_v1.3_fullcleansed.csv').drop(['Unnamed: 0'],axis=1).sample(frac=1.0)
df_test = pd.read_csv('train_test_sets/test_KorCCViD_v1.3_fullcleansed.csv').drop(['Unnamed: 0'],axis=1).sample(frac=1.0)

# df_train = pd.read_csv('KorCCViD_v1.3_fullcleansed.csv').sample(frac=1.0)

In [7]:
df_train.head()

,Transcript,Label
848,근까 그거 고등학교 시즌 시작 그리고 대학교 들어가 시즌 시즌 마다 잖아 니까 당연...,0
194,그래 언니 거기 아들 근데 강아지 강아지 10 10 16 20 그래 면은 그게 얼마...,0
19,졸업 서도 꼐속 으니까 그래서 당연히 간다고 생각 생각 해도 한지 김태희 아예 친해...,0
901,여보세요 수고 십니다 서울 지방 경찰청 경제 범죄 수사 정산 합니다 경제 범죄 수사...,1
748,솔직히 결국 정말 자기 한테 선택 진짜 여러 사례 정말 부럽 직장 다니 다가 진짜 ...,0


In [8]:
# df_test.head()

In [9]:
%%time
# Preprocessing our data
# Convert a collection of raw documents to a matrix of TF-IDF features.
tfidf = TfidfVectorizer(# analyzer='word',
                        ngram_range=(1, 3), #(2, 6), # 3,6
                        # max_df=1, 
                        min_df=2, # 10
                        max_features=2000, #300, 500, 10000 (thsi will limit the vocavulary) 
                        # sublinear_tf=True,
                        # lowercase=False,
                        use_idf=True
                        )
tfidf

CPU times: user 47 µs, sys: 8 µs, total: 55 µs
Wall time: 79.6 µs


In [10]:
# tfidf

# train_test_split

In [11]:
X_train = tfidf.fit_transform(df_train['Transcript']).toarray()
X_test = tfidf.transform(df_test['Transcript']).toarray()

# slip the datasets
y_train, y_test = df_train['Label'], df_test['Label']
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.125, shuffle= True)

# training with fulldataset
# y_train = df_train['Label']
# X_train_sub, X_test, y_train_sub, y_test = train_test_split(X_train, y_train, test_size=0.3, shuffle= True)

# Setting up our results dataframe
df_results = pd.DataFrame(columns=['F1_score', 'Precision', 'Recall', 'Accuracy', 'Accuracy1', 'Training time'])

In [12]:
print(X_train_sub)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.03216573 0.         0.0282444  ... 0.01549669 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [13]:
tfidf.get_feature_names_out()

array(['10', '10 정도', '100', ..., '휴면 계좌', '힘드', '힘들'], dtype=object)

In [14]:
print(X_train.shape)

(974, 2000)


In [15]:
print(X_train_sub.shape)

(852, 2000)


In [16]:
print(X_test.shape)

(244, 2000)


In [17]:
# df_results

In [18]:
print(len(X_train_sub))
# print(len(X_valid))
print(len(X_test))

852
244


In [19]:
# X_test

In [20]:
# print(X_train_sub)
# print(X_valid)
# print(y_train_sub)
# print(y_valid)

# Build the model


In [21]:
#Decision Tree
# dt = DecisionTreeClassifier(max_depth=12, random_state=1234)
# dt

In [22]:
#Random Forest
rf = RandomForestClassifier(n_estimators=500,
                            max_features=0.06,
                            n_jobs=6,
                            random_state=1234)
rf

RandomForestClassifier(max_features=0.06, n_estimators=500, n_jobs=6,
                       random_state=1234)

In [23]:
#Adaboost
# base_estim = DecisionTreeClassifier(max_depth=1, max_features=0.06)                            
# ab = AdaBoostClassifier(base_estimator=base_estim,
#                         n_estimators=500,
#                         learning_rate=0.5,
#                         random_state=1234)
# ab

In [24]:
#Gradient Boosted Decision Trees (Scikit-Learn)
# gbm = GradientBoostingClassifier(n_estimators=2000,
#                                  subsample=0.67,
#                                  max_features=0.06,
#                                  validation_fraction=0.1,
#                                  n_iter_no_change=15,
#                                  verbose=0,
#                                  random_state=1234)
# gbm

In [25]:
#XGBoost
xgb = XGBClassifier(n_estimators=2000,
                    tree_method='hist',
                    subsample=0.67,
                    colsample_level=0.06,
                    verbose=0,
                    n_jobs=6,
                    random_state=1234)
xgb

XGBClassifier(colsample_level=0.06, n_estimators=2000, n_jobs=6,
              random_state=1234, subsample=0.67, tree_method='hist', verbose=0)

In [54]:
#LightGBM
lgbm = LGBMClassifier(n_estimators=2000,
                      feature_fraction=0.06,
                      bagging_fraction=0.67,
                      bagging_freq=1,
                      verbose=0,
                      n_jobs=6,
                      random_state=1234)
lgbm

LGBMClassifier(bagging_fraction=0.67, bagging_freq=1, feature_fraction=0.06,
               n_estimators=2000, n_jobs=6, random_state=1234, verbose=0)

In [27]:
#CatBoost
cb = CatBoostClassifier(n_estimators=2000,
                        colsample_bylevel=0.06,
                        # max_leaves=31,
                        subsample=0.67,
                        thread_count=6,
                        verbose=0,
                        random_state=1234)
cb

In [28]:
cb.get_params()

{'colsample_bylevel': 0.06,
 'n_estimators': 2000,
 'random_state': 1234,
 'subsample': 0.67,
 'thread_count': 6,
 'verbose': 0}

In [29]:
#Bonus: Histogram-based Gradient Boosting Machine (Scikit-Learn)
# hgbm = HistGradientBoostingClassifier(max_iter=2000,
#                                       validation_fraction=0.1,
#                                       n_iter_no_change=15,
#                                       verbose=0,
#                                       random_state=1234)
# hgbm

In [30]:
#Comparing our Competitors
# models = [dt, rf, ab, gbm, hgbm, xgb, lgbm, cb]
models = [rf, xgb, lgbm, cb]
model_names = [i.__class__.__name__ for i in models]

In [31]:
#to specify which of our models require early stopping within the .fit() method.
es_models = ['XGBClassifier',
             'LGBMClassifier',
             'CatBoostClassifier']

# train models

In [32]:
from timeit import default_timer as timer

start = timer()
#Train each of our candidate models
for m, n in zip(models, model_names):
    
    if n in es_models:
        start_time = time()
        m.fit(X_train_sub,
              y_train_sub,
              eval_set = [(X_valid, y_valid)],
              early_stopping_rounds=15,
              verbose=0)
    else:
        start_time = time()
        m.fit(X_train, y_train)
    
    run_time = time() - start_time
    accuracy = np.mean(m.predict(X_test) == y_test)
        
    df_results.loc[n] = [None, None, None, None, accuracy, run_time]
    
    del m
    
train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

The time is 35.7056 seconds


In [33]:
cb.get_all_params()

{'auto_class_weights': 'None',
 'bayesian_matrix_reg': 0.10000000149011612,
 'best_model_min_trees': 1,
 'boost_from_average': False,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS',
 'border_count': 254,
 'class_names': [0, 1],
 'classes_count': 0,
 'depth': 6,
 'eval_metric': 'Logloss',
 'feature_border_type': 'GreedyLogSum',
 'force_unit_auto_pair_weights': False,
 'grow_policy': 'SymmetricTree',
 'iterations': 2000,
 'l2_leaf_reg': 3,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'leaf_estimation_iterations': 10,
 'leaf_estimation_method': 'Newton',
 'learning_rate': 0.02253100089728832,
 'loss_function': 'Logloss',
 'max_leaves': 64,
 'min_data_in_leaf': 1,
 'model_shrink_mode': 'Constant',
 'model_shrink_rate': 0,
 'model_size_reg': 0.5,
 'nan_mode': 'Min',
 'od_pval': 0,
 'od_type': 'Iter',
 'od_wait': 15,
 'penalties_coefficient': 1,
 'pool_metainfo_options': {'tags': {}},
 'posterior_sampling': False,
 'random_seed': 1234,
 'random_strength': 1,
 'rsm': 0.05999999865

In [34]:
df_results

,F1_score,Precision,Recall,Accuracy,Accuracy1,Training time
RandomForestClassifier,NaN,NaN,NaN,NaN,0.983607,4.573857
XGBClassifier,NaN,NaN,NaN,NaN,0.950820,1.589698
LGBMClassifier,NaN,NaN,NaN,NaN,0.991803,2.351793
CatBoostClassifier,NaN,NaN,NaN,NaN,0.971311,26.563245


In [35]:
## Testing LinearSVC
start = timer()

svc = LinearSVC()

start_time = time()
svc.fit(X_train, y_train)
run_time = time() - start_time

preds = svc.predict(X_test)
accuracy = np.mean(preds == y_test)
df_results.loc['LinearSVC'] = [None, None, None, None, accuracy, run_time]

train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

The time is 0.0625 seconds


In [36]:
df_results

,F1_score,Precision,Recall,Accuracy,Accuracy1,Training time
RandomForestClassifier,NaN,NaN,NaN,NaN,0.983607,4.573857
XGBClassifier,NaN,NaN,NaN,NaN,0.950820,1.589698
LGBMClassifier,NaN,NaN,NaN,NaN,0.991803,2.351793
CatBoostClassifier,NaN,NaN,NaN,NaN,0.971311,26.563245
LinearSVC,NaN,NaN,NaN,NaN,0.995902,0.021845


In [37]:
# table = df_results[['accuracy','run_time']].head()

In [38]:
# ax = df_results.plot()
# fig = ax.get_figure()
# fig.savefig('asdf.png')

In [39]:
# ls -l

# Model with recall and precision metrics


## librairies

In [40]:
!pip install psutil nodejs matplotlib seaborn==0.8.1 neptune-client==0.3.7 neptune-contrib[versioning]==0.11.0 neptune-notebooks==0.0.8 scikit-plot

     |████████████████████████████████| 178 kB 6.0 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 829 kB 40.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 181 kB 41.3 MB/s 
     |████████████████████████████████| 276 kB 44.6 MB/s 
     |████████████████████████████████| 2.0 MB 36.5 MB/s 
     |████████████████████████████████| 132 kB 47.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 8.6 MB 36.3 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 127 kB 47.7 MB/s 
     |████████████████████████████████| 130 kB 48.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
  Created wheel for seaborn: filename=seaborn-0.8.1-py3-none-any.whl size=184850 sha256=61ca6d722ba2268c542ad4f207f47dfc4942210b81af1ed3f4c6a29

## Utils.py

In [41]:
import matplotlib.pyplot as plt
import neptune
from neptunecontrib.monitoring.utils import send_figure
import pandas as pd
import scikitplot.metrics as plt_metrics
from scikitplot.helpers import binary_ks_curve
import seaborn as sns
import sklearn.metrics as sk_metrics

plt.rcParams.update({'font.size': 18})
plt.rcParams.update({'figure.figsize': [16, 12]})


def log_binary_classification_metrics(y_true, y_pred, threshold=0.5):
    # Confusion Matrix
    # fig, ax = plt.subplots()
    # plot_confusion_matrix(y_true, y_pred[:, 1] > threshold, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    # plt.close()

    # Plot classification report
    # fig = plot_classification_report(y_true, y_pred[:, 1] > threshold)
    # send_figure(fig, channel_name='metric_charts')
    # plt.close()

    scores = _class_metrics(y_true, y_pred[:, 1], threshold)
    # for metric_name, score in scores.items():
        # print('{} :{}'.format(metric_name, score))
        # neptune.log_metric(metric_name, score)

    # ROC AUC
    roc_auc = sk_metrics.roc_auc_score(y_true, y_pred[:, 1])
    # print('ROC AUC score {}'.format(roc_auc))
    # neptune.log_metric('roc_auc', roc_auc)

    fig, ax = plt.subplots()
    plt_metrics.plot_roc(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # PR AUC
    avg_precision = sk_metrics.average_precision_score(y_true, y_pred[:, 1])
    # print('Average precision score {}'.format(avg_precision))
    # neptune.log_metric('avg_precision', avg_precision)

    fig, ax = plt.subplots()
    plt_metrics.plot_precision_recall(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Brier loss
    brier = sk_metrics.brier_score_loss(y_true, y_pred[:, 1])
    # print('Brier loss {}'.format(brier))
    # neptune.log_metric('brier_loss', brier)

    # Log loss
    log_loss = sk_metrics.log_loss(y_true, y_pred)
    # print('Log loss {}'.format(log_loss))
    # neptune.log_metric('log_loss', log_loss)

    # Kolmogorov Smirnow
    res = binary_ks_curve(y_true, y_pred[:, 1])
    ks_stat = res[3]
    # print('KS stat {}'.format(ks_stat))
    # neptune.log_metric('ks_statistic', ks_stat)

    fig, ax = plt.subplots()
    plt_metrics.plot_ks_statistic(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Cumulative gain
    fig, ax = plt.subplots()
    plt_metrics.plot_cumulative_gain(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Lift curve
    fig, ax = plt.subplots()
    plt_metrics.plot_lift_curve(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Plots by threshold
    figs = plot_metrics_per_threshold(y_true, y_pred[:, 1])

    for fig in figs:
        # send_figure(fig, channel_name='metrics_by_threshold')
        plt.close()

    return scores


def plot_confusion_matrix(y_true, y_pred_class, ax=None):
    cmap = plt.get_cmap('Blues')
    cm = sk_metrics.confusion_matrix(y_true, y_pred_class)
    sns.heatmap(cm, cmap=cmap, annot=True, fmt='g', ax=ax)


def plot_metrics_per_threshold(y_true, y_pred_positive):
    scores_by_thres = _class_metrics_by_threshold(y_true, y_pred_positive)
    figs = []
    for name in scores_by_thres.columns:
        if name == 'threshold':
            continue
        else:
            best_thres, best_score = _get_best_thres(scores_by_thres, name)
            fig, ax = plt.subplots()
            ax.plot(scores_by_thres['threshold'], scores_by_thres[name])
            ax.set_title('{} by threshold'.format(name))
            ax.axvline(x=best_thres, color='red')
            ax.text(x=best_thres, y=0.96 * best_score,
                    s='thres={:.4f}\nscore={:.4f}'.format(best_thres, best_score),
                    color='red')
            figs.append(fig)
    return figs


def plot_classification_report(y_true, y_pred_class):
    report = sk_metrics.classification_report(y_true, y_pred_class, output_dict=True)
    report_df = pd.DataFrame(report).transpose().round(4)

    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=report_df.values,
             colLabels=report_df.columns,
             rowLabels=report_df.index,
             loc='center',
             bbox=[0.2, 0.2, 0.8, 0.8])
    fig.tight_layout()

    return fig


def _class_metrics(y_true, y_pred, threshold):
    y_pred_class = y_pred > threshold

    tn, fp, fn, tp = sk_metrics.confusion_matrix(y_true, y_pred_class).ravel()

    true_positive_rate = tp / (tp + fn)
    true_negative_rate = tn / (tn + fp)
    positive_predictive_value = tp / (tp + fp)
    negative_predictive_value = tn / (tn + fn)
    false_positive_rate = fp / (fp + tn)
    false_negative_rate = fn / (tp + fn)
    false_discovery_rate = fp / (tp + fp)

    scores = {'accuracy': sk_metrics.accuracy_score(y_true, y_pred_class),
              'precision': sk_metrics.precision_score(y_true, y_pred_class),
              'recall': sk_metrics.recall_score(y_true, y_pred_class),
              'f1_score': sk_metrics.fbeta_score(y_true, y_pred_class, beta=1),
              # 'f2_score': sk_metrics.fbeta_score(y_true, y_pred_class, beta=2),
              # 'matthews_corrcoef': sk_metrics.matthews_corrcoef(y_true, y_pred_class),
              # 'cohen_kappa': sk_metrics.cohen_kappa_score(y_true, y_pred_class),
              # 'true_positive_rate': true_positive_rate,
              # 'true_negative_rate': true_negative_rate,
              # 'positive_predictive_value': positive_predictive_value,
              # 'negative_predictive_value': negative_predictive_value,
              # 'false_positive_rate': false_positive_rate,
              # 'false_negative_rate': false_negative_rate,
              # 'false_discovery_rate': false_discovery_rate,
              }

    return scores


def _class_metrics_by_threshold(y_true, y_pred_positive, thres_nr=100):
    thresholds = [i / thres_nr for i in range(1, thres_nr, 1)]

    scores_per_thres = []
    for thres in thresholds:
        scores = _class_metrics(y_true, y_pred_positive, thres)
        scores['threshold'] = thres
        scores_per_thres.append(pd.Series(scores))

    return pd.DataFrame(scores_per_thres)


def _get_best_thres(scores_by_thres, name):
    best_res = scores_by_thres[scores_by_thres[name] == scores_by_thres[name].max()][['threshold', name]]
    position = len(best_res) // 2
    result = best_res.iloc[position].to_dict()
    return result['threshold'], result[name]


## training 2

In [42]:
import lightgbm
import neptune
from neptunecontrib.monitoring.utils import pickle_and_send_artifact
from neptunecontrib.versioning.data import log_data_version
import pandas as pd

# from utils import log_binary_classification_metrics

from timeit import default_timer as timer

start = timer()
#Train each of our candidate models
for m, n in zip(models, model_names):
  if n in es_models:
    start_time = time()
    m.fit(X_train_sub,
          y_train_sub,
          # eval_set = [(X_valid, y_valid)],
          eval_set =[(X_test, y_test)],
          early_stopping_rounds=8,
          verbose=0,
          # eval_metric="logloss"
          )
    run_time = time() - start_time

  else:
    start_time = time()
    m.fit(X_train, y_train)
    run_time = time() - start_time
    
  # Evaluate model
  accuracy = np.mean(m.predict(X_test) == y_test)
  y_test_pred = m.predict_proba(X_test)

  print(n)
  metric_scrores = log_binary_classification_metrics(y_test, y_test_pred)
  # pickle_and_send_artifact((y_test, y_test_pred), 'test_predictions.pkl')
  for metric_name, score in metric_scrores.items():
    print('{} :{}'.format(metric_name, score))

  df_results.loc[n] = [metric_scrores.get("f1_score"),
                       metric_scrores.get("precision"), 
                       metric_scrores.get("recall"), 
                       metric_scrores.get("accuracy"),
                       accuracy,
                       run_time]    
  print('-'*50)
  del m
    
train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

RandomForestClassifier
accuracy :0.9836065573770492
precision :1.0
recall :0.963963963963964
f1_score :0.9816513761467891
--------------------------------------------------
XGBClassifier


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/loc

accuracy :0.9508196721311475
precision :0.9900990099009901
recall :0.9009009009009009
f1_score :0.9433962264150942
--------------------------------------------------
LGBMClassifier
accuracy :0.9918032786885246
precision :1.0
recall :0.9819819819819819
f1_score :0.9909090909090909
--------------------------------------------------
CatBoostClassifier
accuracy :0.9713114754098361
precision :1.0
recall :0.9369369369369369
f1_score :0.9674418604651163
--------------------------------------------------
The time is 33.4272 seconds


In [43]:
## Testing LinearSVC
start = timer()
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# svc = make_pipeline(StandardScaler(),
#                     LinearSVC(probability=True, random_state=0, tol=1e-5))
# svc = LinearSVC()
svc = SVC(kernel='linear', probability=True)

start_time = time()
svc.fit(X_train, y_train)
run_time = time() - start_time

preds = svc.predict(X_test)
y_test_pred = svc.predict_proba(X_test)
accuracy = np.mean(preds == y_test)


# df_results.loc['LinearSVC'] = [score, run_time]

print("LinearSVC")
metric_scrores = log_binary_classification_metrics(y_test, y_test_pred)
# pickle_and_send_artifact((y_test, y_test_pred), 'test_predictions.pkl')
# for metric_name, score in metric_scrores.items():
  # print('{} :{}'.format(metric_name, score))

df_results.loc['LinearSVC'] = [metric_scrores.get("f1_score"),
                                metric_scrores.get("precision"), 
                                metric_scrores.get("recall"), 
                                metric_scrores.get("accuracy"),
                                accuracy,
                                run_time]      
print('-'*50)
del svc

train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

LinearSVC
--------------------------------------------------
The time is 4.1825 seconds


# Training results

In [44]:
df_results

,F1_score,Precision,Recall,Accuracy,Accuracy1,Training time
RandomForestClassifier,0.981651,1.000000,0.963964,0.983607,0.983607,4.012525
XGBClassifier,0.943396,0.990099,0.900901,0.950820,0.950820,1.098843
LGBMClassifier,0.990909,1.000000,0.981982,0.991803,0.991803,2.062703
CatBoostClassifier,0.967442,1.000000,0.936937,0.971311,0.971311,17.496662
LinearSVC,1.000000,1.000000,1.000000,1.000000,0.995902,2.290415


## Save performances result to SVC file


In [45]:
# save the training results
df_results.to_csv('performances_shallow_algorithm_lab_new1.csv')

In [46]:
# ls -l

# Testing the CatBoost text feature

In [47]:
# !pip install ipywidgets

In [48]:
# !jupyter nbextension enable --py widgetsnbextension

In [49]:
# df_train['Transcript'] = df_train['Transcript'].astype(str)
# X_train = df_train.Transcript.tolist()
# len(X_train)

In [50]:
# X_train_text, X_valid_text, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.1)

In [51]:
# # Testing the CatBoost text feature
# model = CatBoostClassifier(n_estimators=2000,
#                            text_features=[0],
#                            colsample_bylevel=0.06,
#                            max_leaves=31,
#                            subsample=0.67,
#                            verbose=0,
#                            thread_count=6,
#                            random_state=1234)
# start = timer()
# model.fit(X_train_text,
#           y_train_sub,
#           eval_set = [(X_valid_text, y_valid)],
#           early_stopping_rounds=15,
#           verbose=0,
#           plot=True)

In [52]:
# train_time = timer() - start
# print('The time is {:.4f} seconds'.format(train_time))